In [5]:
from ultralytics import YOLO
import cv2
import numpy as np

**YOLO (You only look once)**

In [ ]:
# یولو یه الگوریتم دیپ لرنینگه که برای تشخیص اشیا، جدا کردن آبجکت از تصویر، کلاسبندی، تشخیص نقاط کلیدی بدن انسان و دنبال کردن آبجکت در تصاویر و ویدیو ها استفاده میشه
# یولو 8 تا ورژن داره که جدید ترین و بهترینش وی8 هست ولی مشکلش اینه که به پردازش بالاتری نیاز داره
# یولو8 در اندازه های مختلفی طراحی شده که هر چی بزرگتر میشه پیچیده تر میشه ولی پردازش بیشتری هم نیاز داره
# باید با توجه به پروژه بهترین نوع انتخاب بشه
# : به ترتیب از کوچیک به بزرگ
# nano, small, medium, large, x-large
# حرف اول هر کدوم رو بعد از یولو وی8 موقع لود بنویسیم همون دانلود میشه
# تصاویر درست شده توسط یولو توی یه فولدر به اسم رانز ذخیره میشن
# به عنوان سورس ورودی مدل یولو، علاوه بر عکس و ویدیو میتونیم لینک یوتیوب و... هم بدیم

**Detection in image**

In [ ]:
# تشخیص آبجکت های یک تصویر
# دور آبجکت یه مستطیل میکشه و اسم آبجکت و درصد تعلق به کلاس انتخاب شده رو هم نشون میده
# کلا 80 تا کلاس رو تشخیص میده که شامل اشیا مختلفی مثل ماشین، آدم، حیوانات و... هستن

# لود کردن مدل مخصوص دیتکشن یولو با اندازه نانو
detection_model = YOLO('yolov8n.pt')

# save=True : ذخیره کردن عکس
# conf=0.6 : فقط کانفیدنس های بالای 60 درصد رو در نظر بگیر
# save_txt=True : اطلاعات آبجکت هارو توی یه فایل تکست ذخیره کن
# اول کلاس آبجکت که یکی از 80 تاست. بعد اندازه مستطیل دور آبجکت که به صورت نرمالایز هست
# save_conf=True : کانفیدنس هارو هم توی فایل تکست بنویس در آخر
results = detection_model('./images/bicycles.jpg', save=True, conf=0.6, save_txt=True, save_conf=True)

# نمایش تصویر خروجی
predicted_objects_image = cv2.imread('./runs/detect/predict/bicycles.jpg')

cv2.imshow('Predicted objects image', predicted_objects_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

**Detection in video**

In [ ]:
# تشخیص آبجکت های یک ویدیو

detection_model = YOLO('yolov8n.pt')

# save_frames=True : تمام فریم هارو ذخیره کن
# stream=True : به‌ جای پردازش کل ویدیو یه‌جا (که سنگین و کند می‌شه)، مدل رو توی حالت استریم قرار میده. این یعنی فریم‌ها یکی یکی با ژنراتور پردازش میشن که باعث میشه حافظه کمتری مصرف بشه
result_frames = detection_model('./videos/dashcam.mp4', save=True, save_frames=True, stream=True)

# چون از استریم استفاده کردیم، فریم ها یکی یکی میان و ما باید هر بار فریم پردازش شده رو بگیریم
for frame in result_frames:
    pass

# نمایش ویدیو
result_video = cv2.VideoCapture('./runs/detect/predict2/dashcam.avi')

if not result_video.isOpened():
    print('Sorry, the video was not opened.')

while result_video.isOpened():
    ret, frame = result_video.read()

    if ret:
        cv2.imshow('result_video', frame)

    if cv2.waitKey(30) == 27:
        result_video.release()
        break

cv2.destroyAllWindows()

**Segmentation**

In [ ]:
# جدا کردن آبجکت از تصویر

# لود کردن مدل مخصوص سگمنتیشن
segmentation_model = YOLO('yolov8n-seg.pt')

results = segmentation_model('./images/bicycles.jpg', save=True)

# نمایش
semented_objects_image = cv2.imread('./runs/segment/predict/bicycles.jpg')

cv2.imshow('Segmented objects image', semented_objects_image)
# نمایش ماسک ها

# اول برای درک بهتر ایکس و وای ها یا همون مختصات ماسک رو نشون میدیم
# results[0] : چون مشخصات داخل یه لیستن و ما عنصر اولشو میخوایم
# xy[0] : ماسک آبجکت سگمنت شده ااول
print(results[0].masks.xy[0])

# نمایش خود ماسک
mask = results[0].masks.data[0]
print(mask)
# حالا باید ماسک رو به یه آرایه تبدیل کنیم چون تصویر یه آرایست نه لیست
mask = mask.numpy()

# الان میخوایم آبجکت رو روی ماسک نشون بدیم
# اول ماسک رو هم سایز تصویرمون میکنیم
image = cv2.imread('./images/bicycles.jpg')
resized_mask = np.uint8(cv2.resize(mask, (image.shape[1], image.shape[0]))) # نوع داده ماسک باید حتما یواینت8 باشه

# بعد با عملگر اند آبجکت رو روی ماسک میندازیم
object_segmented_image = cv2.bitwise_and(image, image, mask=resized_mask)

# نمایش
cv2.imshow('Original', image)
cv2.imshow('Mask of first object', mask)
cv2.imshow('First object segmented image', object_segmented_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\images\bicycles.jpg: 480x640 4 persons, 2 bicycles, 283.3ms
Speed: 6.7ms preprocess, 283.3ms inference, 26.5ms postprocess per image at shape (1, 3, 480, 640)
[[      147.2       529.2]
 [      147.2         502]
 [      145.6       500.4]
 ...
 [      156.8       519.6]
 [      156.8         526]
 [      150.4       532.4]]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


**Classification**

In [ ]:
# کلاسیندی آبجکت های توی تصویر
# توی کلسیفیکیشن یولو، ما 1000 تا کلاس داریم به جای 80 تا
# اینطوریه که آبجکت اصلی تصویر تشحیص داده میشه و بعد 5 تا کلاسی که بیشترین کانفیدنس رو دارن نمایش داده میشن

classification_model = YOLO('yolov8n-cls.pt')

result = classification_model('./images/dog.jpeg', save=True)

# نمایش
classified_object_image = cv2.imread('./runs/classify/predict/dog.jpg')

cv2.imshow('Classified object image', classified_object_image)

# تشخیص کلاس با بیشترین کانفیدنس
best_confidence_index = result[0].probs.top1
best_confidence_name = result[0].names[best_confidence_index]
print(f'Index of the best confidenced class: {best_confidence_index} | Name of the class: {best_confidence_name}')

cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\images\dog.jpeg: 224x224 German_shepherd 0.94, malinois 0.02, keeshond 0.02, groenendael 0.01, Australian_terrier 0.00, 29.5ms
Speed: 5.3ms preprocess, 29.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Index of the best confidenced class: 235 | Name of the class: German_shepherd


**Pose estimation**

In [121]:
# تشخیص نقاط کلیدی بدن انسان توی تصویر
# هفده تا نقطه کلیدی تشخیص داده میشن که اگه یه وجود نداشته باشه یا خارج از تصویر باشه مقدار غایب بهش تعلق میگیره

pose_estimation_model = YOLO('yolov8n-pose')

results = pose_estimation_model('./images/people.webp', save=False)

# شیپ نقاط کلیدی
print(results[0].keypoints.xy.shape) # اولی تعداد انسان ها، بعدش تعداد نقاط کلیدی که همیشه 17 تاست و بعدم مختصات نقطه
# نمایش مختصات نقاط کلیدی
key_points = results[0].keypoints.xy.numpy()
print(key_points)

# نمایش
people_pose_estimated_image = cv2.imread('./runs/pose/predict/people.jpg')

# cv2.imshow('People pose estimated image', people_pose_estimated_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\images\people.webp: 640x640 3 persons, 320.9ms
Speed: 15.2ms preprocess, 320.9ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)
torch.Size([3, 17, 2])
[[[     625.76      91.293]
  [     639.82      77.528]
  [     613.46      77.866]
  [     665.64      80.512]
  [     600.06      81.999]
  [     714.33      160.08]
  [      576.9      154.51]
  [     771.34      251.93]
  [     532.94      251.72]
  [     739.54       278.1]
  [     522.37       343.4]
  [     687.53      377.24]
  [     596.91      368.95]
  [     680.07      542.51]
  [     563.89      532.81]
  [     667.58      682.02]
  [     546.96      682.28]]

 [[     405.95      97.065]
  [     418.74      83.567]
  [      393.2      84.252]
  [     437.74      87.904]
  [     375.43      90.073]
  [     468.52      171.64]
  [      339.7      161.39]
  [     475.66      276.56]
  [     306.79      236.86]
  [  

**Tracking**

In [125]:
# دنبال کردن آجکت ها توی ویدیو

model = YOLO('yolov8n.pt')

# show=True : همزمان با پردازش فریم ها اونارو نمایش هم بده
# persist=True : یعنی به هر آبجکت یه آیدی مخصوص بده و اون رو تو حافظه نگه دار تا اگه اون آبجکت دوباره تو ویدیو ظاهر شد بفهمی همونه
# اگه فالس بدیم آیدی توی حافظه ذخیره نمیشه و تغییر میکنه
result_frames = model.track('./videos/dashcam.mp4', save=True, show=True, persist=True)


WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/63) c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\videos\dashcam.mp4: 384x640 2 cars, 1 truck, 302.4ms
video 1/1 (frame 2/63) c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\videos\dashcam.mp4: 384x640 2 cars, 320.3ms
video 1/1 (frame 3/63) c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\videos\dashcam.mp4: 384x640 2 cars, 21

**Fine-Tuning**

In [ ]:
# استفاده از دیتاست متفرقه برای تشخیص یا کلاسبندی یه آبجکت خاص
# از اونجایی که کلاس های دیتکشن کلا 80تاست و ما اغلب اوقات به کلاس های بیشتری نیاز داریم، میتونیم از دیتاست های جدا برای ترین یولو و استفاده ازش استفاده کنیم
# دلیل اینکه یولو به طور دیفالت کلا 80 تا برای دیتکشن داره اینه که دیتکشن نیاز به داده های پیچیده مثل کادر و... داره
# دیتکشن باید بهمون بگه توی تصویر چیا هست و هرچی کجاست؟
# ولی کلسیفیکیشن فقط میگه تصویر در مورد چیه؟ یه دونه چی توی تصویر هست؟ به خاطر همین ساده تره و 1000 تا کلاس داره

# ترین کردن دیتاست روی یولو
model = YOLO('yolov8n.pt')
# imgsz=640 : تصاویر قبل از ورود به مدل ریسایز میشن
fine_tuning = model.train(data='./datasets/solar_panels_detection/data.yaml', epochs=20, batch=8, device='cpu', imgsz=640)

# استفاده از مدل ترین شده
solar_panels_detection_model = YOLO('./runs/detect/train/weights/best.pt')
result = solar_panels_detection_model('./datasets/solar_panels_detection/test/images/DJI_0753_MP4-28_jpg.rf.33506bc0ca65dd9508a746ae3e514942.jpg', save=True)

# نمایش
solar_panels_detected = cv2.imread('./runs/detect/predict3/DJI_0753_MP4-28_jpg.rf.33506bc0ca65dd9508a746ae3e514942.jpg')
solar_panels_detected = cv2.resize(solar_panels_detected, (0, 0), fx=0.4, fy=0.4)

cv2.imshow('Solar panels detected', solar_panels_detected)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 c:\Users\Abolfazl\Desktop\computer\programming\machine_learning\computer_vision\yolo\datasets\solar_panels_detection\test\images\DJI_0753_MP4-28_jpg.rf.33506bc0ca65dd9508a746ae3e514942.jpg: 384x640 3 solar-panelss, 144.8ms
Speed: 7.9ms preprocess, 144.8ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
